In [1]:
import gensim
import logging
import os
import re
import string
import itertools
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np

from gensim import corpora, models, similarities

In [2]:
corpus = [[(0, 1.0), (1, 1.0), (2, 1.0)],
           [(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (8, 1.0)],
           [(1, 1.0), (3, 1.0), (4, 1.0), (7, 1.0)],
           [(0, 1.0), (4, 2.0), (7, 1.0)],
           [(3, 1.0), (5, 1.0), (6, 1.0)],
           [(9, 1.0)],
           [(9, 1.0), (10, 1.0)],
           [(9, 1.0), (10, 1.0), (11, 1.0)],
           [(8, 1.0), (10, 1.0), (11, 1.0)]]

tfidf = models.TfidfModel(corpus)
vec = [(0, 1), (4, 1)]
tfidf[vec]

[(0, 0.8075244024440723), (4, 0.5898341626740045)]

### Strings to Vectors

In [9]:
documents = ["Human machine interface for lab abc computer applications",
              "A survey of user opinion of computer system response time",
              "The EPS user interface management system",
              "System and human system engineering testing of EPS",
              "Relation of user perceived response time to error measurement",
              "The generation of random binary unordered trees",
              "The intersection graph of paths in trees",
              "Graph minors IV Widths of trees and well quasi ordering",
              "Graph minors A survey"]

In [27]:
# remove stopwords
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
        for document in documents]

# remove word that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
        for text in texts]

dictionary = corpora.Dictionary(texts)
print(dictionary.token2id)

new_doc = 'Human computer interaction with human human human'
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

{u'minors': 11, u'graph': 10, u'system': 6, u'trees': 9, u'eps': 8, u'computer': 1, u'survey': 5, u'user': 7, u'human': 2, u'time': 4, u'interface': 0, u'response': 3}
[(1, 1), (2, 4)]


In [32]:
# BagWords
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('test.mm', corpus) # save to file for later use
corpus

[[(0, 1), (1, 1), (2, 1)],
 [(1, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(0, 1), (6, 1), (7, 1), (8, 1)],
 [(2, 1), (6, 2), (8, 1)],
 [(3, 1), (4, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(5, 1), (10, 1), (11, 1)]]

### Transformation

In [37]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(1, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.44424552527467476), (6, 0.3244870206138555), (7, 0.3244870206138555)]
[(0, 0.5710059809418182), (6, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(2, 0.49182558987264147), (6, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (4, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(5, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


#### LSI

In [39]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)
corpus_lsi = lsi[corpus_tfidf]
lsi.print_topics(2)

[(0,
  u'0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  u'-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [40]:
for doc in corpus_lsi:
    print(doc)
# lsi.save('file_to_save.lsi')
# lsi.load('file_to_load.lsi')

[(0, 0.066007833960902804), (1, -0.52007033063618502)]
[(0, 0.19667592859142299), (1, -0.76095631677000519)]
[(0, 0.089926399724463077), (1, -0.72418606267525143)]
[(0, 0.075858476521780557), (1, -0.63205515860034334)]
[(0, 0.10150299184979941), (1, -0.57373084830029586)]
[(0, 0.70321089393783165), (1, 0.16115180214025668)]
[(0, 0.87747876731198393), (1, 0.16758906864659256)]
[(0, 0.90986246868185872), (1, 0.14086553628718854)]
[(0, 0.6165825350569285), (1, -0.053929075663894835)]


### Similiarity Queries

In [42]:
doc = 'Human computer interaction'
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi) # vec_lsi order by similiarity to vec_bow

[(0, 0.079104751174447582), (1, -0.57328352430794027)]


# Beatles test

In [19]:
folder = './../data/beatles/'
num_topics = 5
import random

def read_data(folder):
    texts, names = [], []
    filenames = filter(lambda it: it.endswith('.txt'), os.listdir(folder))
    for fname in filenames:
        names.append(fname.replace('.txt', ''))
        with open(folder + fname, 'r') as f:
            texts.append(re.sub(r'\[.*\]', '', f.read().replace('\n',' ').strip()))
    return names, texts

def prepare_data(records):
    return [[word for word in record.translate(None, string.punctuation).lower().split() if word not in STOPWORDS]
        for record in records]

def create_lsi(dictionary, corpus, num_topics=5):
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=num_topics)
    return lsi_model

def cread_lda(dictionary, corpus, num_topics=5, passes=20, chunksize=2000):
    number_of_topics = 5
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    lda_model = models.ldamodel.LdaModel(corpus_tfidf, id2word=dictionary,
                                   num_topics=number_of_topics,
                                   passes=passes,
                                   chunksize=chunksize)
    return lda_model

def get_top_words(lda_model):
    # http://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html
    top_words = [[word for word, __ in lda_model.show_topic(topic, topn=50)] for topic in range(lda_model.num_topics)]
    replacements = []
    for words in top_words:
        yield words

In [19]:
song_names, raw_texts = read_data(folder)
texts = prepare_data(raw_texts)
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [20]:
lsi_model = create_lsi(dictionary, corpus)
for t, topic in lsi_model.print_topics(num_topics):
    print('Topic {0} : {1}'.format(t, topic))

Topic 0 : -0.402*"love" + -0.213*"yeah" + -0.213*"ill" + -0.206*"im" + -0.164*"dont" + -0.157*"baby" + -0.150*"know" + -0.144*"girl" + -0.142*"want" + -0.140*"oh"
Topic 1 : 0.706*"love" + -0.169*"yeah" + -0.166*"baby" + -0.159*"john" + 0.132*"need" + 0.131*"whoa" + -0.117*"im" + 0.115*"true" + -0.115*"got" + -0.104*"brian"
Topic 2 : 0.523*"john" + 0.380*"brian" + 0.219*"paul" + 0.191*"yeah" + 0.182*"love" + -0.150*"dont" + 0.147*"beatles" + 0.128*"ha" + -0.121*"let" + 0.114*"play"
Topic 3 : 0.455*"yeah" + -0.346*"girl" + -0.231*"blackbird" + -0.203*"moment" + -0.165*"fly" + -0.158*"john" + 0.142*"ill" + -0.134*"waiting" + -0.132*"arise" + 0.131*"oh"
Topic 4 : 0.389*"baby" + -0.263*"blackbird" + -0.200*"yeah" + -0.194*"fly" + -0.175*"waiting" + -0.160*"moment" + 0.155*"girl" + -0.152*"night" + 0.151*"man" + -0.150*"arise"


In [22]:
random.seed(42)

lda_model = cread_lda(dictionary, corpus, num_topics)
for t, top_words in lda_model.print_topics(num_topics=num_topics, num_words=5):
    print("Topic", t, ":", top_words)

('Topic', 0, ':', u'0.004*ill + 0.003*yeah + 0.003*love + 0.003*goodbye + 0.002*im')
('Topic', 1, ':', u'0.004*let + 0.003*im + 0.003*baby + 0.002*dance + 0.002*come')
('Topic', 2, ':', u'0.003*love + 0.003*gonna + 0.003*dont + 0.002*long + 0.002*dear')
('Topic', 3, ':', u'0.004*love + 0.003*loving + 0.002*johnny + 0.002*blackbird + 0.002*moment')
('Topic', 4, ':', u'0.005*love + 0.003*yeah + 0.003*dont + 0.003*im + 0.003*want')


In [9]:
# Получение темы для конкретного документа
for i in range(20):
    print(song_names[i], lda_model[corpus[i]])

('1822!', [(1, 0.52211132048712006), (3, 0.45661616145185091)])
('A Day In The Life', [(0, 0.93234032464485683), (3, 0.064800838710947758)])
("A Hard Day's Night", [(2, 0.59639866382676765), (3, 0.401130343938853)])
('A Little Rhyme', [(2, 0.84725434711504188), (3, 0.14571822990448782)])
('A Shot Of Rhythm And Blues', [(2, 0.66083510545051505), (3, 0.33722872941850446)])
('A Taste Of Honey', [(0, 0.99277053712726293)])
('Across The Universe', [(2, 0.030486131292410516), (4, 0.96649820511462992)])
('Act Naturally', [(3, 0.47476828978075786), (4, 0.52233039100101319)])
("Ain't She Sweet", [(2, 0.71058409092586106), (3, 0.28601270684145963)])
("All I've Got To Do", [(2, 0.24092343960557047), (3, 0.75527546340440466)])
('All My Loving', [(3, 0.46507086697440275), (4, 0.53085947506280862)])
('All Things Must Pass', [(3, 0.20633866267931289), (4, 0.78962031495826934)])
('All Together Now', [(0, 0.99638830713218329)])
('All You Need Is Love', [(3, 0.9964525033033127)])
('And I Love Her', [(3,

### Other test. Nightwish

In [20]:
folder = './../data/nightwish/'
song_names, raw_texts = read_data(folder)
texts = prepare_data(raw_texts)
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [5]:
lsi_model = create_lsi(dictionary, corpus)
for t, topic in lsi_model.print_topics(num_topics):
    print('Topic {0} : {1}'.format(t, topic))

Topic 0 : 0.196*"wish" + 0.132*"love" + 0.125*"heart" + 0.121*"world" + 0.110*"time" + 0.108*"come" + 0.107*"oh" + 0.103*"night" + 0.100*"long" + 0.094*"eyes"
Topic 1 : 0.381*"wish" + 0.238*"angel" + 0.167*"miss" + -0.134*"drown" + 0.131*"im" + 0.124*"colder" + 0.124*"getting" + 0.112*"right" + -0.108*"ago" + 0.107*"wings"
Topic 2 : -0.181*"walks" + -0.172*"forever" + -0.171*"heaven" + -0.160*"heart" + 0.147*"right" + -0.144*"walk" + -0.125*"meadows" + 0.125*"crestfallen" + -0.117*"little" + -0.109*"wish"
Topic 3 : -0.219*"heaven" + -0.173*"meadows" + 0.148*"horizon" + 0.130*"world" + -0.127*"cradle" + 0.126*"away" + 0.124*"end" + 0.121*"far" + 0.111*"ago" + 0.109*"long"
Topic 4 : -0.176*"right" + -0.162*"night" + -0.159*"things" + 0.156*"wheres" + -0.153*"fight" + -0.118*"land" + -0.116*"horizon" + -0.115*"lost" + -0.109*"crestfallen" + 0.107*"dont"


In [6]:
random.seed(42)

lda_model = cread_lda(dictionary, corpus, num_topics)
for t, top_words in lda_model.print_topics(num_topics=num_topics, num_words=5):
    print("Topic", t, ":", top_words)

('Topic', 0, ':', u'0.002*wheres + 0.001*star + 0.001*wish + 0.001*ones + 0.001*thou')
('Topic', 1, ':', u'0.002*away + 0.002*save + 0.002*phantom + 0.002*opera + 0.002*wish')
('Topic', 2, ':', u'0.002*angel + 0.002*bye + 0.002*wish + 0.001*nighttime + 0.001*crownless')
('Topic', 3, ':', u'0.002*come + 0.002*slow + 0.002*drown + 0.002*love + 0.002*heart')
('Topic', 4, ':', u'0.002*meadows + 0.002*heaven + 0.001*yearn + 0.001*bless + 0.001*gone')


In [16]:
for topic, words in enumerate(get_top_words(lda_model)):
    print('Toplic {0} : {1}\n'.format(topic, ' '.join(words[:10])))

Toplic 0 : wheres star wish ones thou ticktock rain caress ago neverfading

Toplic 1 : away save phantom opera wish lost eva killing world know

Toplic 2 : angel bye wish nighttime crownless elvenpath revealed wheel tied fate

Toplic 3 : come slow drown love heart rest silent got leave time

Toplic 4 : meadows heaven yearn bless gone read neverneverland beautiful remember wandering

